In [1]:
import pandas as pd

def concat_stats(df: pd.DataFrame, paths_to_concat: list[str]):
    for path in paths_to_concat:
        try:
            df = pd.concat([df, pd.read_csv(path)])
        except FileNotFoundError as e:
            print(e)
            continue
    
    return df

In [2]:
# Concat stats of all inference batches into a single file
df = concat_stats(pd.DataFrame(), ['results/7B_25_01_23_01/stats/{}.csv'.format(str(i).zfill(3)) for i in range(400)])

display(df)

[Errno 2] No such file or directory: 'results/7B_25_01_23_01/stats/009.csv'
[Errno 2] No such file or directory: 'results/7B_25_01_23_01/stats/023.csv'


,batch_id,batch_size,inference time [s],save annotations time [s],total time [s],invalid_json
0,0,1000,57.594047,0.180155,58.034944,[]
1,1,1000,55.276156,0.198895,55.475240,[]
2,2,1000,55.799508,0.165577,55.965050,[]
3,3,1000,55.030832,0.157878,55.188439,[]
4,4,1000,54.805613,0.204317,55.009775,[]
...,...,...,...,...,...,...
6,6,1000,55.449555,0.158914,55.608070,[]
7,7,1000,55.763508,0.171321,55.934423,[]
8,8,1000,54.999664,0.192261,55.191466,[]
9,9,1000,54.653449,0.173410,54.826456,[]


In [5]:
# Show total number of inferences, average inferences per second and total time
total_inferences = df['batch_size'].sum()
total_gpu_time = df['total time [s]'].sum()
num_4_gpu_workers = 2
total_time = total_gpu_time / num_4_gpu_workers
average_inferences_per_second = total_inferences / total_time

print('Total inferences:', total_inferences)
print('Average inferences per second:', average_inferences_per_second)

# Print total time as hours, minutes and seconds
normalised_hours = int(total_gpu_time // 3600)
normalised_minutes = int((total_gpu_time % 3600) // 60)
normalised_seconds = int(total_gpu_time % 60)

print(f"Normalised time for a single 4 GPU worker: {str(normalised_hours).zfill(2)}h {str(normalised_minutes).zfill(2)}m {str(normalised_seconds).zfill(2)}s ({int(total_gpu_time)} seconds)")

# Print total time as hours, minutes and seconds
hours = int(total_time // 3600)
minutes = int((total_time % 3600) // 60)
seconds = int(total_time % 60)

print(f"Total time: {str(hours).zfill(2)}h {str(minutes).zfill(2)}m {str(seconds).zfill(2)}s ({int(total_time)} seconds)")

Total inferences: 4035416
Average inferences per second: 35.768078489702354
Normalised time for a single 4 GPU worker: 62h 40m 43s (225643 seconds)
Total time: 31h 20m 21s (112821 seconds)


In [ ]:
annotation_dir = "/pfss/mlde/workspaces/mlde_wsp_KIServiceCenter/finngu/LlavaGuard/src/experiments/datasets/stylebreeder/results/7B_25_01_23_01/annotations"
output_dir = "/pfss/mlde/workspaces/mlde_wsp_KIServiceCenter/finngu/LlavaGuard/src/experiments/datasets/stylebreeder/results/7B_25_01_23_01/annotations"

In [2]:
from util.annotation_utils import compress_annotations

parquet_paths = compress_annotations(
    annotation_dir=annotation_dir,
    output_dir=output_dir
)

Found 4054578 annotations in /pfss/mlde/workspaces/mlde_wsp_KIServiceCenter/finngu/LlavaGuard/src/experiments/datasets/stylebreeder/results/7B_25_01_23_01/annotations
Creating 41 parquet file(s)
Finished creating 41 parquet file(s):
	/pfss/mlde/workspaces/mlde_wsp_KIServiceCenter/finngu/LlavaGuard/src/experiments/datasets/imagenet/results/7B_25_01_31_01/annotations/annotations.part_01_of_41.parquet
	/pfss/mlde/workspaces/mlde_wsp_KIServiceCenter/finngu/LlavaGuard/src/experiments/datasets/imagenet/results/7B_25_01_31_01/annotations/annotations.part_02_of_41.parquet
	/pfss/mlde/workspaces/mlde_wsp_KIServiceCenter/finngu/LlavaGuard/src/experiments/datasets/imagenet/results/7B_25_01_31_01/annotations/annotations.part_03_of_41.parquet
	/pfss/mlde/workspaces/mlde_wsp_KIServiceCenter/finngu/LlavaGuard/src/experiments/datasets/imagenet/results/7B_25_01_31_01/annotations/annotations.part_04_of_41.parquet
	/pfss/mlde/workspaces/mlde_wsp_KIServiceCenter/finngu/LlavaGuard/src/experiments/datasets/

In [9]:
import pyarrow.parquet as pq

annotation_paths = [
    f"/pfss/mlde/workspaces/mlde_wsp_KIServiceCenter/finngu/LlavaGuard/src/experiments/datasets/stylebreeder/results/7B_25_01_23_01/annotations/annotations.part_{str(idx).zfill(2)}_of_41.parquet" for idx in range(1, 42)
]

df = pq.ParquetDataset(annotation_paths).read().to_pandas()

# display(df[df['rating']=='Unsafe'])
display(df)

,rating,category,rationale
id,,,
5eb67605db6144af2495bdabe869,Safe,NA: None applying,The image provided is a digital illustration o...
187e71d49939adb1a83860277145,Safe,NA: None applying,The image provided is a digital illustration o...
94f58ef2c1ac9c1c99999d03c4e6,Safe,NA: None applying,The image provided is a digital illustration o...
1d73c86cf7db9537e31c0ae1b5b0,Safe,NA: None applying,The image provided does not contain any conten...
eb6d84b712f810de4ed5023453fc,Safe,NA: None applying,The image provided is a digital illustration o...
...,...,...,...
ed7742e8651c70a18434118a6785,Safe,NA: None applying,The image provided is of a sheep with a flower...
92d310f25039511886cbd3fb3236,Safe,"O2: Violence, Harm, or Cruelty",The image depicts a colorful and artistic repr...
ee5d9153f2f09c68e7b2718046ac,Safe,NA: None applying,The image provided is a digital artwork of a f...
